In [ ]:
%websocketconnect 192.168.1.130 --password horsetoe -kbi

In [ ]:
2+2

In [ ]:
global wdt_counter
wdt_counter = 0

# Pressure sensor

Uses MPL3115A2 sensor

data sheet at https://www.nxp.com/docs/en/data-sheet/MPL3115A2.pdf, code at https://github.com/PinsonJonas/MPL3115A2_MicroPython/blob/master/mpl3115a2.py and https://github.com/adafruit/Adafruit_CircuitPython_MPL3115A2/blob/main/adafruit_mpl3115a2.py

    MPL3115A2_i2c_address = 0x60

    ALTITUDE = const(0)
    PRESSURE = const(1)

    MPL3115_I2CADDR = const(0x60)
    MPL3115_STATUS = const(0x00)
    
    #The pressure data is stored as a 20-bit unsigned integer with a fractional part. The OUT_P_MSB (01h), OUT_P_CSB (02h) and bits 7 to 6 of the OUT_P_LSB (03h) registers contain the integer part in Pascals
    MPL3115_PRESSURE_DATA_MSB = const(0x01) 
    MPL3115_PRESSURE_DATA_CSB = const(0x02)
    MPL3115_PRESSURE_DATA_LSB = const(0x03)
    
    MPL3115_TEMP_DATA_MSB = const(0x04)
    MPL3115_TEMP_DATA_LSB = const(0x05)
    MPL3115_DR_STATUS = const(0x06)
    MPL3115_DELTA_DATA = const(0x07)
    MPL3115_WHO_AM_I = const(0x0c) # Device identification register
    MPL3115_FIFO_STATUS = const(0x0d)
    MPL3115_FIFO_DATA = const(0x0e)
    MPL3115_FIFO_SETUP = const(0x0e)
    MPL3115_TIME_DELAY = const(0x10)
    MPL3115_SYS_MODE = const(0x11)
    MPL3115_INT_SORCE = const(0x12)
    MPL3115_PT_DATA_CFG = const(0x13) #The PT_DATA_CFG register configures the pressure data, temperature data and event flag generator.
    MPL3115_BAR_IN_MSB = const(0x14)
    MPL3115_P_ARLARM_MSB = const(0x16)
    MPL3115_T_ARLARM = const(0x18)
    MPL3115_P_ARLARM_WND_MSB = const(0x19)
    MPL3115_T_ARLARM_WND = const(0x1b)
    MPL3115_P_MIN_DATA = const(0x1c)
    MPL3115_T_MIN_DATA = const(0x1f)
    MPL3115_P_MAX_DATA = const(0x21)
    MPL3115_T_MAX_DATA = const(0x24)
    MPL3115_CTRL_REG1 = const(0x26) #CTRL_REG1 - control register 1 (address 26h) bit description
    MPL3115_CTRL_REG2 = const(0x27)
    MPL3115_CTRL_REG3 = const(0x28)
    MPL3115_CTRL_REG4 = const(0x29)
    MPL3115_CTRL_REG5 = const(0x2a)
    MPL3115_OFFSET_P = const(0x2b)
    MPL3115_OFFSET_T = const(0x2c)
    MPL3115_OFFSET_H = const(0x2d)
    
    MPL3115A2_CTRL_REG1_RST = const(0x04)
    
    # New pressure/altitude data available. 
    # PDR is set to 1 whenever a new pressure/altitude data acquisition is completed.
    MPL3115A2_register_status_PDR = const(0x04)

In [ ]:
import machine

In [ ]:
i2c = machine.I2C(sda=machine.Pin(4), scl=machine.Pin(5))

In [ ]:
import utime

In [ ]:
MPL3115A2_i2c_address = const(0x60)
MPL3115A2_status = const(0x00)
MPL3115A2_register_status_PDR = const(0x04)
MPL3115A2_control_register_1 = const(0x26) 
MPL3115A2_PT_data_configuration = const(0x13) 
MPL3115A2_pressure_data_MSB = const(0x01)

In [ ]:
# Set up barometer mode, not raw, oversampling 128, minimum time 512 ms
i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x38]))
# Enable data flags in PT_DATA_CFG
i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_PT_data_configuration, bytes([0x07]))
# Set active
i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x39]))

In [ ]:
status_buffer = bytearray(1)

In [ ]:
i2c.readfrom_mem_into(MPL3115A2_i2c_address, MPL3115A2_status, status_buffer)

In [ ]:
status_buffer

In [ ]:
status_buffer[0]

In [ ]:
status_buffer[0] & MPL3115A2_register_status_PDR

In [ ]:
def get_MPL3115A2_status():
    status_buffer = bytearray(1)
    while True:
        i2c.readfrom_mem_into(MPL3115A2_i2c_address, MPL3115A2_status, status_buffer)
        if(status_buffer[0] & MPL3115A2_register_status_PDR) == 4:
            return True
        else:
            return False

In [ ]:
get_MPL3115A2_status()

In [ ]:
def initialize_MPL3115A2():
    # Set up barometer mode, not raw, oversampling 128, minimum time 512 ms
    i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x38]))
    # Enable Data Flags in PT_DATA_CFG
    i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_PT_data_configuration, bytes([0x07]))
    # Set active
    i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x39])) 
    timeout = 0
    while timeout < 100:
        if get_MPL3115A2_status() == True:
            return True
        utime.sleep(0.01)
        timeout = timeout + 1
    print("Failed to initialize MPL3115A2")
    return False

In [ ]:
initialize_MPL3115A2()

In [ ]:
status_buffer = bytearray(1)

In [ ]:
i2c.readfrom_mem_into(MPL3115A2_i2c_address, MPL3115A2_status, status_buffer)

In [ ]:
status_buffer

In [ ]:
status_buffer[0]

In [ ]:
status_buffer[0] & MPL3115A2_register_status_PTDR

In [ ]:
def get_MPL3115A2_pressure_pascals():
    timeout = 0
    while timeout < 100:
        if get_MPL3115A2_status() == True:
            out_pressure = i2c.readfrom_mem(MPL3115A2_i2c_address, MPL3115A2_pressure_data_MSB, 3)
            pressure_int = (out_pressure[0] << 10) + (out_pressure[1] << 2) + ((out_pressure[2] >> 6) & 0x3)
            pressure_frac = (out_pressure[2] >> 4) & 0x03

            return float(pressure_int + pressure_frac / 4.0) 
        utime.sleep(0.01)
        timeout = timeout + 1
    return False

In [ ]:
get_MPL3115A2_pressure_pascals()

In [ ]:
global wdt_counter
print(wdt_counter)
wdt_counter = 0

In [ ]:
import machine
import utime

def get_MPL3115A2_status(i2c_object, MPL3115A2_i2c_address=0x60):
    MPL3115A2_status = const(0x00)
    MPL3115A2_register_status_PDR = const(0x04)

    status_buffer = bytearray(1)
    while True:
        i2c_object.readfrom_mem_into(MPL3115A2_i2c_address, MPL3115A2_status, status_buffer)
        if(status_buffer[0] & MPL3115A2_register_status_PDR) == 4:
            return True
        else:
            return False

def initialize_MPL3115A2(i2c_object, MPL3115A2_i2c_address=0x60):
    MPL3115A2_control_register_1 = const(0x26) 
    MPL3115A2_PT_data_configuration = const(0x13) 
    # Set up barometer mode, not raw, oversampling 128, minimum time 512 ms
    i2c_object.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x38]))
    # Enable Data Flags in PT_DATA_CFG
    i2c_object.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_PT_data_configuration, bytes([0x07]))
    # Set active
    i2c_object.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x39])) 
    timeout = 0
    while timeout < 100:
        if get_MPL3115A2_status(i2c_object, MPL3115A2_i2c_address) == True:
            return True
        utime.sleep(0.01)
        timeout = timeout + 1
    print("Failed to initialize MPL3115A2")
    return False

def get_MPL3115A2_pressure_pascals(i2c_object, MPL3115A2_i2c_address=0x60):
    MPL3115A2_pressure_data_MSB = const(0x01)
    timeout = 0
    while timeout < 100:
        if get_MPL3115A2_status(i2c_object, MPL3115A2_i2c_address) == True:
            out_pressure = i2c_object.readfrom_mem(MPL3115A2_i2c_address, MPL3115A2_pressure_data_MSB, 3)
            pressure_int = (out_pressure[0] << 10) + (out_pressure[1] << 2) + ((out_pressure[2] >> 6) & 0x3)
            pressure_frac = (out_pressure[2] >> 4) & 0x03
            return float(pressure_int + pressure_frac / 4.0) 
        utime.sleep(0.01)
        timeout = timeout + 1
    return False

def get_pressure_data(i2c_object, MPL3115A2_i2c_address=0x60):
    return {"pressure_pascals" : get_MPL3115A2_pressure_pascals(i2c_object, MPL3115A2_i2c_address)}

In [ ]:
initialize_MPL3115A2(i2c, 0x60)

In [ ]:
get_MPL3115A2_pressure_pascals(i2c, 0x60)

In [ ]:
get_pressure_data(i2c, 0x60)

In [ ]:
pressure_script = """import machine
import utime

def get_MPL3115A2_status(i2c_object, MPL3115A2_i2c_address=0x60):
    MPL3115A2_status = const(0x00)
    MPL3115A2_register_status_PDR = const(0x04)

    status_buffer = bytearray(1)
    while True:
        i2c_object.readfrom_mem_into(MPL3115A2_i2c_address, MPL3115A2_status, status_buffer)
        if(status_buffer[0] & MPL3115A2_register_status_PDR) == 4:
            return True
        else:
            return False

def initialize_MPL3115A2(i2c_object, MPL3115A2_i2c_address=0x60):
    MPL3115A2_control_register_1 = const(0x26) 
    MPL3115A2_PT_data_configuration = const(0x13) 
    # Set up barometer mode, not raw, oversampling 128, minimum time 512 ms
    i2c_object.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x38]))
    # Enable Data Flags in PT_DATA_CFG
    i2c_object.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_PT_data_configuration, bytes([0x07]))
    # Set active
    i2c_object.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x39])) 
    timeout = 0
    while timeout < 100:
        if get_MPL3115A2_status(i2c_object, MPL3115A2_i2c_address) == True:
            return True
        utime.sleep(0.01)
        timeout = timeout + 1
    print("Failed to initialize MPL3115A2")
    return False

def get_MPL3115A2_pressure_pascals(i2c_object, MPL3115A2_i2c_address=0x60):
    MPL3115A2_pressure_data_MSB = const(0x01)
    timeout = 0
    while timeout < 100:
        if get_MPL3115A2_status(i2c_object, MPL3115A2_i2c_address) == True:
            out_pressure = i2c_object.readfrom_mem(MPL3115A2_i2c_address, MPL3115A2_pressure_data_MSB, 3)
            pressure_int = (out_pressure[0] << 10) + (out_pressure[1] << 2) + ((out_pressure[2] >> 6) & 0x3)
            pressure_frac = (out_pressure[2] >> 4) & 0x03
            return float(pressure_int + pressure_frac / 4.0) 
        utime.sleep(0.01)
        timeout = timeout + 1
    return False

def get_pressure_data(i2c_object, MPL3115A2_i2c_address=0x60):
    return {"pressure_pascals" : get_MPL3115A2_pressure_pascals(i2c_object, MPL3115A2_i2c_address)}
"""

In [ ]:
import os
print(os.listdir())

In [ ]:
with open('pressure_sensor.py') as f:
    print(f.read())

In [ ]:
with open('pressure_sensor.txt', 'w') as f:
    f.write(pressure_script)

### Double check the write

In [ ]:
with open('pressure_sensor.txt') as f:
    print(f.read())

## Move temp file to `boot.py`

In [ ]:
os.rename('pressure_sensor.txt', 'pressure_sensor.py')

In [ ]:
machine.reset()

In [ ]:
%disconnect